<a href="https://colab.research.google.com/github/sebacea/AnEco2/blob/master/ETLRegCivilSII19862Fecu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Activación R magic
%load_ext rpy2.ipython

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
#lectura de archivos y homogenización de Nombre y Rut
%%R
library(readxl)
library(tidyverse)

# Registro 19862
databasicos <- read_excel("/content/drive/My Drive/COS/datos basicos.xlsx")
datacontacto <- read_excel("/content/drive/My Drive/COS/contacto.xlsx")
datacargos <- read_excel("/content/drive/My Drive/COS/cargos.xlsx")
# SII
dataSII <- read_excel("/content/drive/My Drive/COS/nomina_empresas_pjuridicas_at2018.xlsx", skip = 2)
# FECU Social
fecu <- read.csv("/content/drive/My Drive/COS/FECU.csv")

names(fecu)[names(fecu) == "Nombre.organización"] <- "Nombre Organización"
names(databasicos)[names(databasicos) == "Nombre"] <- "Nombre Organización"

names(fecu)[names(fecu) == "RUT"] <- "Rut Organizacion"
names(dataSII)[names(dataSII) == "RUT"] <- "Rut Organizacion"

databasicos$"Rut Organizacion"<-gsub("\\.", "" , databasicos$"Rut Organizacion")
databasicos$"Rut Organizacion"<-gsub("\\-", "" , databasicos$"Rut Organizacion")

datacontacto$"Rut Organizacion"<-gsub("\\.", "" , datacontacto$"Rut Organizacion")
datacontacto$"Rut Organizacion"<-gsub("\\-", "" , datacontacto$"Rut Organizacion")

datacargos$"Rut Organizacion"<-gsub("\\.", "" , datacargos$"Rut Organizacion")
datacargos$"Rut Organizacion"<-gsub("\\-", "" , datacargos$"Rut Organizacion")

fecu$"Rut Organizacion"<-gsub("\\.", "" , fecu$"Rut Organizacion")
fecu$"Rut Organizacion"<-gsub("\\-", "" , fecu$"Rut Organizacion")

dataSII$"Rut Organizacion"<-paste(dataSII$"Rut Organizacion",dataSII$DV, sep="")

/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: ✔ ggplot2 3.2.1     ✔ purrr   0.3.3
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0

  warnings.warn(x, RRuntimeWarning)
/usr/local/lib/python3.6/dist-packages/rpy2/rinterface/__init__.py:146: RRuntimeWarning: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

  warnings.warn(x, RRuntimeWarning)


#### **BD Registro civil (df2)**

Se filtran datos del Registro Civil generando marca de superación de pobreza

In [4]:
%%R
data <- read.csv2("/content/drive/My Drive/COS/RegistroCivilUC.csv")

df <-data[which(data$TIPO %in% c("CORPORACION", "CORPORACION/FUNDACION", "FUNDACION", "")),]

#df2 corresponde a los datos presentes en la pestaña "1er filtro" del excel
df2 <- df[which(df$CLASIFICACION %in% c("AGRUPACION DE CULTURA","AGRUPACION DE DISCAPACIDAD","AGRUPACION DE MUJERES","AGRUPACION DE SALUD", "AGRUPACION DE SEGURIDAD",
                                        "AGRUPACION DE VIVIENDA","AGRUPACION ETNICA","AGRUPACION FOLCLORICA","AGRUPACION JUVENIL","AGRUPACION SOCIAL","ANCIANOS",
                                        "ASISTENCIA SOCIAL","CENTRO ADULTO MAYOR","CLUB DE DAMAS","CLUB DEPORTIVO","CULTURALES","DEPORTIVAS","EDUCACION","N/D","NIÑOS",
                                        "ONG","OTRAS","RELIGIOSAS","SALUD","SERVICIO BIENESTAR","UNION COMUNAL","")),]


#SUPERACIÓN DE LA POBREZA

df2$SUP.POBREZA<-ifelse(grepl("AGRUPACION FOLCLORICA",df2$CLASIFICACION),'NO',NA)

# Filtros aplicados a los nombres de las organizaciones:
df2$SUP.POBREZA<-ifelse(grepl("CUERPO DE BOMBEROS",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("MUNICIPAL",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("MUNICIPALES",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("MUNICIPALIDAD",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CENTRO CULTURAL",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CLUB",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CIRCULO",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("MUTUAL",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("IGLESIA",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("MUSEO",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("LIGA DEPORTIVA",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("DEPORTIVA LIGA",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CUERPO DE OFICIALES",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CUERPO DE SUBOFICIALES",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("HERMANDAD",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("SOCORROS MUTUOS",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CENTRO DE EXALUMNOS",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CENTRO DE EXALUMNAS",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CENTRO DE EX ALUMNOS",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CENTRO DE EX ALUMNAS",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CENTRO DE EX - ALUMNOS",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CENTRO DE EX - ALUMNAS",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CENTRO DE EX-ALUMNOS",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CENTRO DE EX-ALUMNAS",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("VETERINARIO",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("VETERINARIOS",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("JUNTA DE VECINOS",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("OFICIALES",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("SUBOFICIALES",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CARABINEROS",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("BOMBEROS",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CATEQUESIS",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA) 
df2$SUP.POBREZA<-ifelse(grepl("SERVICIO DE BIENESTAR DEL PERSONAL",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("SERVICIO DE SALUD DEL PERSONAL",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("SERVICIO DE BIENESTAR DE LOS TRABAJADORES",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA) 
df2$SUP.POBREZA<-ifelse(grepl("SERVICIO DE BIENESTAR DEL MAGISTERIO",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA) 
df2$SUP.POBREZA<-ifelse(grepl("BIENESTAR DE TRABAJADORES",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("ASOCIACION CLARO BIENESTAR",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA) 
df2$SUP.POBREZA<-ifelse(grepl("CORPORACION DE BIENESTAR DE ESSEL",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("SERVICIO BIENESTAR SOCIAL TRABAJADORES",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("BIENESTAR DE",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CENTRO GENERAL",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("EX ",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CENTRO DE PADRES",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CENTRO DE PADRE",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CENTRO DE VECINOS",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CENTRO DE PROPAGANDA",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CENTRO DE DAMAS",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CENTRO DE AMIGOS DEL LICEO",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CENTRO DE AMIGOS DE LA ESCUELA",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CENTRO DE AMIGOS DE ESTUDIANTES",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CENTRO DE MADRES",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CENTRO DE MAESTRAS",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CENTRO DE MINEROS",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CENTRO DE HIJOS",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("ANIMAL",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("ANIMALES",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("PERRO",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("GATO",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CABALLO",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("MASCOTA",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("SINDICATO",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("COMITE",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("COMITÉ",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA) 
df2$SUP.POBREZA<-ifelse(grepl("EVANGELISTICO",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("EJERCITO",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("MILITAR",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("ARMADA",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("NAVAL",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CAMARA",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CÁMARA",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("FRATERNIDAD UNIVERSITARIA",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("FRATERNIDAD ECOLOGICA UNIVERSITARIA",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("FRATERNIDAD CABALLEROS DE LA MAGIA",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("ASOCIACION CENTRAL",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("COOPERATIVA CENTRAL",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("PENSION",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("JUBILA",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA) 
df2$SUP.POBREZA<-ifelse(grepl("FONDO DE INDEMNIZACIONES",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("INSTITUTO",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("ASOCIACION DE PADRES",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("ASOCIACION DE EX-ALUMNOS",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("ASOCIACION DE EX ALUMNOS",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("ASOCIACION INSTITUTO",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("SCOUT",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("UNION SOCIAL",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CARGADOR",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CONSEJO",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("VEGETAL",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CAJA DE",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("FUERZAS",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("SOCIEDAD DE LA SAGRADA FAMILIA DE LA PARROQUIA DE LA ASUNCION",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("ASUNCION",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("ASOCIACIÓN DEL PERSONAL",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("EMPLEADO",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("SOCORRO",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("COOPERATIVA",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("RETIRO",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("MINERA",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("VETERANO",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("GREMIAL",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("GREMIO",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("ASOCIACION COMUNAL",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("ASOCIACION DEPORTIVA",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("ARBITR",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CONTADOR",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("FEDERACION",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)



df2$SUP.POBREZA<-ifelse(grepl("FUNDACION",df2$NOMBRE.PJ),'SI',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("FUNDACIÓN",df2$NOMBRE.PJ),'SI',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("LIGA PROTECTORA",df2$NOMBRE.PJ),'SI',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("LIGA CONTRA",df2$NOMBRE.PJ),'SI',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("LIGA DE ESTUDIANTES",df2$NOMBRE.PJ),'SI',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CORPORACION PRIMERA INFANCIA DE LA CAMARA CHILENA DE LA CONSTRUCCION COPRIN",df2$NOMBRE.PJ),'SI',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("FUNDA CIO PARA EL DESARROLLO Y EMPLEO FRATERNIDAD",df2$NOMBRE.PJ),'SI',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("COLONIAS",df2$NOMBRE.PJ),'SI',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CORPORACION DE ORQUESTAS VECINALES U O.N.G MUSICA PARA TODOS",df2$NOMBRE.PJ),'SI',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("ORGANIZACION NO GUBERNAMENTAL DE DESARROLLO ORQUESTA NACIONAL DE CIEGOS DE CHILE",df2$NOMBRE.PJ),'SI',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CENTRO NACIONAL DE MEDIACION, ARBITRAJE Y CONCILIACION",df2$NOMBRE.PJ),'SI',df2$SUP.POBREZA)



#Revisar si corresponden a Superación de la pobreza:
df2$SUP.POBREZA<-ifelse(grepl('SOCIEDAD PROTECTORA DE EMPLEADOS',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL Y DEPORTIVA JOVENES EN UNA DIRECCION',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL DE IQUIQUE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL JOYAS NATIVAS DE TARAPACA U ONG JOYAS NATIVAS DE TARAPACA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL DE ANTOFAGASTA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('SOCIEDAD PROTECTORA DE EMPLEADOS DE TALTAL',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('SOCIEDAD PROTECTORA DE EMPLEADOS DE ANTOFAGASTA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION CULTURAL PARA EL TANGO ARRABALEROS 2000 DE ANTOFAGASTA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION CULTURAL TRADICIONES DE CHILE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION CULTURAL DE LA MUNICIPALIDAD DE MEJILLONES',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION CULTURAL DE LAMUNICIPALIDAD DE TALTAL',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION CULTURAL DE LA MUNICIPALIDAD DE OLLAGUE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION CULTURAL DE LA MUNICIPALIDAD DE MARIA ELENA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION MUJERES TEJEDORAS DE SUEÑOS',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL Y DE TURISMO DE CALAMA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION CULTURAL DE LA MUNICIPALIDAD DE TOCOPILLA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION MUNICIPAL DE DEPORTES DE TALTAL',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL A TEMPO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION MUJERES GUERRERAS DE ANTOFAGASTA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL LA TOMA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION MUNICIPAL DE DESARROLLO COMUNAL Y FOMENTO PRODUCTIVO DE TOCOPILLA, O FUNDACION DE FOMENTO DE TOCOPILLA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL TEATRO EN TIEMPOS DE GUERRA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL SACO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL HIJOS DEL SOL',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACIONCULTURAL ARTISTICA ENTRE OLAS CARRIZAL BAJO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL RAICES DE ATACAMA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION JUNTOS POR UNA VIVIENDA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION PASAJE FRESIA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION ARTESANAL Y CULTURAL LAS TABLAS',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION DEPORTIVA CULTURAL Y RECREATIVA ANT-PUKEM',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION CULTURAL Y DEPORTIVA LAS GOLONDRINAS',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION ARTESANAL Y CULTURAL PACUL',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL BIENAL DE ARTE DE LA SERENA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL Y DE DESARROLLO BODEGON DE LOS VILOS',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION RUNACAY CENTRO LATINOAMERICANO DE ESTUDIOS SOCIALES',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CENTRO SOCIAL CULTURAL CAPACIDADES DIFERENTES INCLUSIVAS CADIN DE LA COMUNA DE COQUIMBO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACIÓN ARTISTICA VOCES DEL VIENTO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION CREANDO ARTE SALAMANCA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL CANELA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL LITERARIA LOS VILOS',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL PERGOLA SAN MIGUEL DE COQUIMBO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL VISION REGIONAL',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL MUCH MUSIC',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL FRIJOL MAGICO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL DIAGUITA ILLARI',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL DE PUTAENDO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL DE LA QUINTA REGION',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('VOLUNTARIAS PARA LA SALUD Y GUARDERIA INFANTIL DE LA COMUNA DE LIMACHE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('SOCIEDAD DE SEÑORAS CATALINA VAN BUREN DE BERISSO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL DEL ALMENDRAL',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION TEATRO MUSEO DEL TITERE Y EL PAYASO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CENTRO DE DAMAS UN SUEÑO REALIZADO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CENTRO PARA EL PROGRESO DE ELRETIRO QUILPUE.',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('SOCIEDAD PROTECTORA DE EMPLEADOSDE ADUANA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('SOCIEDAD PROTECTORA DE EMPLEADOSDE VI?A DEL MAR',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('SOCIEDAD PROTECTORA DE EMPLEADOSDE VALPARAISO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('SOCIEDAD PROTECTORA DE EMPLEADOSDE COMERCIO DE SAN FELIPE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('SOCIEDAD PROTECTORA DE EMPLEADOSDE VALPARAISO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL DE CARTAGENA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACIÓN CULTURAL ALLIANCE FRANCAISE DE ISLA DE PASCUA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CENTRO CULTURAL Y SOCIAL PENSAR CHILE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION CLUB DE TEATRO VIÑA DEL MAR',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL PARA EL FOMENTO DE LAS ARTES, EL PATRIMONIO Y EL URBANISMO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CENTRO CULTURAL DE SUSTENTABILIDAD PIO PIO, PIO PIO, PIO PIO CHILE, CULTURAL CENTER FOR SUSTAINABILITY',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL FESTIVAL COSTUMBRISTA DE LOS ANDES',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL POR LA MUSICA Y EL ARTE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL VERNACULA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('TALLER FEMENINO SAN ALBERTO HURTADO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL Y DEPORTIVA STREET RACING CLUB DE CALLE LARGA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION RURAL FEMENINA DE DEPORTES QUILLOTA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION DE LIDERES ADULTO MAYOR REGION VALPARAISO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION CULTURAL GENARO TORRES',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CENTRO CULTURAL URBANO CHILENITO JIMENEZ',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL CHILETNO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL ASCENSOR CULTURAL',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL QUILPUE O CC QUILPUE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL PRIMERA DEL SUR',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION CULTURAL MUNICIPAL DE OLMUE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION MUNICIPAL DE DEPORTES DE ZAPALLAR',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL RAPANUI',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION MUNICIPAL DE DESARROLLO SOCIAL DE QUILLOTA O FUNDACION BANAMOR O BANAMOR',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION MUNICIPAL DE CULTURA DE ZAPALLAR',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL EL ESCAPARATE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ORGANIZACION NO GUBERNAMENTAL DE DESARROLLO CENTRO DE ESTUDIOS Y DESARROLLO SOCIAL CLUB ZONTA VIÑA DEL MAR CHILE O CLUB ZONTA VIÑA DEL MAR CHILE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION ARTISTICO CULTURAL LOS FAROS',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION MUNICIPAL PARA EL DESARROLLO DE GRANEROS',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION DE CULTURA DE SANTA CRUZ',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION CULTURAL Y SOCIAL ETAREO VILLA VICENTE SANMARTIN',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ESCUELA DE FUTBOL SANTA JULIA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL Y FILANTROPICA HERMANOS FRATERNALES',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL LUCHADORES DEL EMPRENDIMIENTO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION MUNICIPAL PARA EL DESARROLLO CULTURAL DE RENGO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION MUSEO PUEBLITO CHEPICANO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL MADRE SELVA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('GRUPO DE MUJERES FLORECER',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL CREANDO SUEÑOS DE RENGO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL ORIGENES',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ORGANIZACION NO GUBERNAMENTAL DE DESARROLLO MUSEO CAMPESINO EN MOVIMIENTO U ONG MUCAM',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL DE LINARES',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL VIÑAS DEL VALLE DE CURICO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL EL CANARIO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL AGRUPACION CULTURAL EL TRUENO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACIONMUSEO DE LAS ABEJAS',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL VOX LUMINI',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL ESTUDIO V',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL SEGUIDORES DEL TEATRO PROVINCIAL DE CURICO O SE TEATRO CURICO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('SOCIEDAD PROTECTORA DE EMPLEADOS DE TALCAHUANO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL DE SAN PEDRO DE LA PAZ',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CENTRO DE AMIGAS DEL HOSPITAL DECONCEPCION',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CENTRO DE AMIGAS DE PREVENTORIOLAS TOTORITAS.',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CENTRO DE AMIGOS DEL HOSPITAL',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL, SOCIAL Y DEPORTIVA GOLFO DE ARAUCO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL DE TANGO CULTURA INCLUSION Y SALUD',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL CHARLES PEGUY',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION CULTURAL JOCA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION DE LOCATARIOS DEL MAR AL PALADAR',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION CULTURAL ALTO VOLTAJE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ORGANIZACION DE ACCION SOCIAL Y CULTURAL CLUB DEPORTIVO PEQUEÑOS TALENTOS MUNICIPALES',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL FUERTE SAN DIEGO DE TUCAPEL',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL Y EDUCACIONAL NESSAGARA O CORPORACION CULTURAL NESSAGARA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL ESCUELA DE MUSICA CONCEPCION',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL Y SOCIAL AMOJEDEL',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION CULTURAL CORRAL JARA MENQUE TOME',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL RUSA DE CONCEPCION',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION PARA EL FOMENTO DEL DEPORTE CLUB BC O FUNDACION CLUB BC',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL Y SOCIAL ESMERALDA O ASOCIACION CULTURAL ESMERALDA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL LO PEQUEN',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL ARTE 21 O C.C. ARTE 21',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL RAKIZUAM',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL COLECTIVO SUR',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL DE ESCRITORES JUAN EMAR',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION CULTURAL JUVENIL UNION HIP HOP NEWEN',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL CENTRO DE ESTUDIOS E INVESTIGACION DE FENOMENOS ANOMALOS DE CHILE TEMUKO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION CULTURAL Y DE MEDICINA ANCESTRAL INDIGENA FILL LAWEN',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION MUSEO ESTEBAN CANATA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION MUSEO NAUTICO NAUTILUS',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL ALBOREA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL LA FABRICA DEL ARTE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL DE OSORNO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL SEMANAS MUSICALES DE FRUTILLAR',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL DOCTOR MATIAS YURASZECK',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION CULTURAL Y SOCIAL AYUN ÑUKE TUE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION CULTURAL FUN BUS',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION JUVENIL GENERACION DE LUZ Y ESPERANZA (GELE)',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION CULTURAL ESTRELLITA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION CULTURAL Y SOCIAL ADMAPU',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION SOCIAL Y CULTURAL DE PADRES Y AMIGOS SINFONICA DE NIÑOSY JOVENES DE QUELLON',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION ARTISTICA, CULTURAL Y CORO RENACER',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL TURISTICA ARTISTICA DEPORTIVA Y SOCIAL ACTADYS PUERTO MONTT',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL SOCIAL KENOSIS CONOCER FORMAR Y CREAR',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL BORDE AUSTRAL',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION MUSEO ANTONIO FELMER NIKLITSCHEK O FUNDACION MUSEO ALEMAN DE NUEVA BRAUNAU',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL KYMCHE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL MUSIC&ART',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL ESPAI COLONA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION EDUCACIONAL CLUB DE LEONES DE COYHAIQUE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ORGANIZACION NO GUBERNAMENTAL DE DESARROLLO LIGA DE LA JUSTICIA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION CULTURAL DE FONDEROS Y RAMADEROS DE COCHRANE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL ESTETICA MASIVA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION DE LA SOCIEDAD CHILENA DE ENFERMEDADES CATASTROFICAS Y DE FAMILIAS EN RIESGO SOCIAL O FUNDACION FRANCISCO LORCA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL AGRUPACION MUSICAL VOCES DEL SUR DEL MUNDO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL ISAIAS',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL AL SUR DEL MAIPO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL, BALMACEDA DOCE QUINCE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL IDENTIDAD PATRIMONIAL',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL DE LA CAMARA CHILENA DE LA CONSTRUCCION',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CENTRO NACIONAL DEL PATRIMONIO FOTOGRAFICO CORPORACION CULTURAL',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL MOISES MUSSA BATTAL',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION CLUB JOSE MANUEL BALMACEDA O FUNDACION PRESIDENTE BALMACEDA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION DE DEPORTES DE LA CAMARA CHILENA DE LA CONSTRUCCION',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION DE CAPACITACION DE LA CAMARA NACIONAL DE COMERCIO DE CHILE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL DE TALAGANTE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL DE PEÑALOLEN',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION SOCIEDAD CHILENA DE PEDIATRIA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION PARQUE Y CENTRO CULTURAL BOTANICO CHAGUAL',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL DE ESTACION CENTRAL',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL MATUCANA CIEN',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL TALLER 99 DE GRABADO NEMESIO ANTUNEZ',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CENTRO DE AMIGOS DEL HOSPITAL DE NIÑOS EXEQUIEL GONZALEZ CORTES',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL CHILETODOS',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL, EDUCACIONAL Y SOCIAL GALILEO GALILEI',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION CASA-MUSEO SABIO IGNACIO DOMEYKO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION DE PATINAJE DE PUENTE ALTO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL MOSAICO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL ORATES',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL BUTACA NARANJA O BUTACA NARANJA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CENTRO DE DIA ADULTOS MAYORES CHILENOS ESPAñOLES UNIDOS POR SIEMPRE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION MUSEO DE LA MEMORIA Y LOS DERECHOS HUMANOS',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION TEATRO MUNICIPAL DE LAS CONDES',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL DE CALERA DE TANGO CODECA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION MUSEO DE LA MODA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL DEL HOSPITALDEL SALVADOR',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL DE RECOLETA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL EDUCARTE CHILEDENOMINADA TAMBIÉN PORSU SIGLA COEDUCARTE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL VITACURA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL DE LA ESTACION MAPOCHO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION EDUCACIONAL DEL ROTARY CLUB DE SANTIAGO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL CORO SAN JOSE DE LA SIERRA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL FOTOESPACIO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACIÓN CULTURAL DE LA ANTIGUA Y PRIMITIVA TRADICIÓN',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION DE MUJERES OMILEN ANTU (RAYOS DE SOL)',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACIÓN LOCAL ARTE CONTEMPORANEO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL ABC CULTURA Y EDUCACION',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL Y RECREATIVA DEL PERSONAL',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL VRAPA VALORAR RESPALDAR APOYAR PROTEGER ARTE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL NACIONAL DE LAS ARTES',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL CARTA ABIERTA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION PEDALEABLE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL LAICA DE LAS CONDES',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION DE LA SOCIEDAD CHILENA DE NEFROLOGIA O FUNDACION PRO SALUD RENAL',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL CHASKY MILLANTUN',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION DE DESARROLLO DEPORTIVO SOCIAL Y CULTURAL ACADEMIA NACIONAL FOOTBALL CHILE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL PARA LA MUSICA Y LAS ARTES DE ÑUÑOA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACIóN DE DESARROLLO CULTURAL NUESTRA SEñORA DEL PERPETUO SOCORRO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACIÓN CEACHEIMUSEO DE LA MEMORIA DEPORTIVACHILE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION DE CULTURA Y PATRIMONIO DE INDEPENDENCIA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION MUSEO GUERRA DEL PACIFICO DOMINGO DE TORO HERRERA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL PORVENIR DE CHILE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL ISLA DE MAIPO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION MUNICIPAL DE CULTURA DE PROVIDENCIA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL CULTURA DE LA BASURA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL COLECTIVO LAS NIÑAS',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL CYMATICA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL DIECIOCHO 173',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL Y DE DESARROLLO SOCIAL ACTIVA EDUCA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL PAISAJE SONORO DE CHILE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL Y SOCIAL PATRIMONIO DE LA FAMILIA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL Y DE DESARROLLO SOCIAL MUNDO NUEVO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL ESPACIO CREAMUNDOS',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL LA CAÑADILLA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL AMBIENTAL TRAITRAICO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL TEATRO CHATARRITAS',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL CULTURA EN ACCION',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION DE BIENESTAR Y SALUD',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL JORGE PEÑA HEN CORPORACION CULTURAL GUITARRAS DE AMERICA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL DE CICLISTAS URBANOS INDEPECLETA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL NUEVA JUVENTUD',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION MUSEO VIOLETA PARRA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION MUSEO DEL TRANSPORTE PATRIMONIAL E HISTORICO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ONG CENTRO DE FORMACION DE FUTBOLISTAS CHILE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL C.M.E.SANTIAGO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL A BORDEREAU',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL DEL PATRIMONIO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACIÓN TEATRAL Y CULTURAL ZAPATOS BLANCOS',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL DE DESARROLLO DE LA MUSICA LAS ARTES ESCENICAS Y OTRO TIPO DE EXPRESIONES ARTISTICAS ARTE Y CREACION',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL SOCIAL Y DEPORTIVA ANTAUCO FESTFOLK',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION CULTURAL ARAMANDO DELIRIO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL FRATERNIDAD COPIAPO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL GONDOR ANORIEN',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL COLECTIVO POETICO LETRAS EN BORGOÑA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ORGANIZACION NO GUBERNAMENTAL DE DESARROLLO BOYS SCOUTS HUALLE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ORGANIZACION NO GUBERNAMENTAL DE DESARROLLO COOPERATIVA OSCAR HEIREMANS SOCIOS FUNDADORES U ONG OSCAR HEIREMANS',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL CHILENO CHINA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL LIBERARTE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION DE BIENESTAR DESARROLLO Y CAPACITACIONCOBIDEC',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CULTURA DEPORTE Y RECREACION LA FRATERNIDAD',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION MUSEO DE LA LITURGIA PAPA BENEDICTO XVI',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL LONGITUD 70',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL VIARTES',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL TRIMAGEN',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION MUSEO CASA DE LA ACUARELA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL LA SERENA DIECISEIS DE OCTUBRE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL PERU DANZA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL SIENTO POR CIENTO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL TEATRO EDUCA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL DE LETHIERES DE CHILE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION MUSEO TALLER',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL TRAMA O CORPORACION CULTURAL TRAMA O CORPORACION TRAMA O TRAMA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL DEPORTIVA Y FOLCLORICA COFOCHI',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL DE MAIPU',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CENTRO CULTURAL ESCUELA DE MUSICA DE LO ESPEJO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL REACCION',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION MUSEO CAMPESINO DE LIRAY O FUNDACION MUCALI',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL TALLER ESTUDIO 112 O TALLER ESTUDIO 112',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL SOCIAL DEPORTIVA TERRA NOVA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL DEL SERENO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL JEAN SIBELIUS',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL Y PATRIMONIAL DE SAN BERNARDO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CASA DE LA FRATERNIDAD',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CENTRO CULTURAL LUCHAR Y CRECER',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION MUSEO TEMPORAL',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION MUSEO DEL CIRCO CHILENO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL RIO CLARILLO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL EL RACO DE PIRQUE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION PROGRAMA DE INVESTIGACION EN MEDICINA TRADICIONAL Y ACUPUNTURA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION BIENESTAR CPVLC',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION EDUCACIONAL CLUB HIPICO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION BORJIANOS',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL Y EDUCACIONAL MINKA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL ARIETE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL CENTRO DE ARTES CASA ARBOL',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL LA GRULLA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL Y SOCIAL PEÑAFLOR',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL CHARANGOS DEL MUNDO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION MUSEO VIRTUAL',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION EPICENTRO CULTURAL',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL URBANA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION MUSEO DE LA PELUQUERIA FAUNDEZ',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL CAMINAR',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL SENDEROS DE EL MONTE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL ISLAMICA DE CHILE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL ORTODOXA SAN BASILO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL CASA PARCAS O CASA PARCAS',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL TEATRO SUR',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL CHILENA ASIATICA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('FUNDACION MUSEO DEL ASCENSOR',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL PAIS EN BLANCO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL ABRECAMINOS',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('SOCIEDAD PROTECTORA DE EMPLEADOS DE COMERCIO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION DE DIABETICOS E HIPERTENSOS A.D.H.',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('AGRUPACION CULTURAL AMIGOS DE LA MUSICA - VALDIVIA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL MEMORIAS DE ISLA TEJA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL COMISION BICENTENARIO DE RIO BUENO',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('SERVICIO DE BIENESTAR DE EDUCACION O SERBIEDUC',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('SOCIEDAD PROTECTORA DE EMPLEADOSDE ARICA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL DE PUEBLOS ORIGINARIOS DE PUTRE CHUNGARA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL LOS VIEJOS ESTANDARTES DE ARICA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CENTRO CULTURAL CIRCUITO AZAPA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL MARKA AMAUTA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL NUMISMATICA DE ARICA',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CENTRO TERAPEUTICO COMUNITARIO THANICHYI',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('ASOCIACION CULTURAL CIRCULO ANCESTRAL VIVENCIAS DE ARTE MEMORIA Y APRENDIZAJE',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl('CORPORACION CULTURAL DE SAN FABIAN',df2$NOMBRE.PJ),'REVISAR',df2$SUP.POBREZA)



df2$SUP.POBREZA<-ifelse(grepl("ANULADA",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("CANCELADA",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)
df2$SUP.POBREZA<-ifelse(grepl("DISUELTA",df2$NOMBRE.PJ),'NO',df2$SUP.POBREZA)    

#Cambio de nombre de las columnas Nombre y Rut
names(df2)[names(df2) == "NOMBRE.PJ"] <- "Nombre Organización"
names(df2)[names(df2) == "RUT"] <- "Rut Organizacion"

#Creación de columna "Reportes de Fecu"
df2$"Reportes de Fecu"<-ifelse(df2$'Rut Organizacion' %in% fecu$'Rut Organizacion','SI',NA)

#Considerar en superación de la pobreza a los que poseen fecu
df2$SUP.POBREZA<-ifelse(grepl("SI",df2$"Reportes de Fecu"),'SI',df2$SUP.POBREZA)


nrow(df2)

[1] 23006


####  **BD Ley 19862**

In [5]:
#Se obtiene un df considerando solo con la primera fila de la tabla de contacto de cada organización
%%R
infocontacto <- datacontacto[which(datacontacto$"#"==1),]
#Merge: datos básicos e infocontacto
ley19862<-merge(databasicos,infocontacto,by="Rut Organizacion",all.x=TRUE)

nrow(ley19862)


[1] 20027


#### **Merge por Rut: df2 - BD Ley 19862**

In [12]:
%%R
df2_ley19862_rut <- merge(df2,ley19862,by="Rut Organizacion")

nrow(df2_ley19862_rut)


[1] 1087


#### **Merge por Nombre: df2 - BD Ley 19862**

In [10]:
%%R
df2_ley19862_nom <- merge(df2,ley19862,by="Nombre Organización")

nrow(df2_ley19862_nom)

[1] 616


In [17]:
%%R
df3<-df2_ley19862_rut[which(df2_ley19862_rut$'Nombre Organización.x' %in% df2_ley19862_nom$'Nombre Organización'),]
nrow(df3)

[1] 264


In [18]:
%%R
df2_ley19862<-df2_ley19862_rut
df2_ley19862$'Rut Organizacion'<-ifelse(df2_ley19862_rut$'Nombre Organización.x' %in% df2_ley19862_nom$'Nombre Organización',df2_ley19862_nom$'Rut Organizacion.y',df2_ley19862$'Rut Organizacion')

SyntaxError: ignored

#### **Merge por Rut: df2 - BD Ley 19862 - SII**

In [19]:
%%R
df2_ley19862_SII_rut <- merge(df2_ley19862_rut,dataSII,by="Rut Organizacion")
nrow(df2_ley19862_SII_rut)

[1] 1087


#### **Merge por Nombre: df2 - BD Ley 19862 - SII**

In [20]:
%%R
df2_ley19862_SII_nom <- merge(df2_ley19862_nom,dataSII,by.x="Nombre Organización",by.y='Razón Social')
nrow(df2_ley19862_SII_nom)


[1] 419
